In [1]:
from notebook.auth import passwd

In [2]:
passwd()

Enter password: ········
Verify password: ········


'sha1:dd9c3eec230c:867942c0e532050e3e3def134cd9c18245508e25'

In [3]:
from IPython.display import HTML

HTML("<iframe src=https://www.quantopian.com/posts/olmar-implementation-fixed-bug width=1100 height=350></iframe>")

In [1]:
import zipline

STOCKS = ['AMD', 'CERN', 'COST', 'DELL', 'GPS', 'INTC', 'MMM']

class OLMAR(zipline.TradingAlgorithm):
    def initialize(self, eps=1, window_length=5):
        self.stocks = STOCKS
        self.m = len(self.stocks)
        self.price = {}
        self.b_t = np.ones(self.m) / self.m
        self.last_desired_port = np.ones(self.m) / self.m
        self.eps = eps
        self.init = True
        self.days = 0
        self.window_length = window_length
        self.add_transform(zipline.transforms.MovingAverage, 'mavg', ['price'],
                           window_length=window_length)
        
        self.set_slippage(zipline.finance.slippage.FixedSlippage())
        self.set_commission(zipline.finance.commission.PerShare(cost=0))

    def handle_data(self, data):
        self.days += 1
        if self.days < self.window_length:
           return
        
        if self.init:
            self.rebalance_portfolio(data, self.b_t)
            self.init = False
            return

        m = self.m

        x_tilde = np.zeros(m)
        b = np.zeros(m)

        # find relative moving average price for each security
        for i, stock in enumerate(self.stocks):
            price = data[stock].price
            # Relative mean deviation
            x_tilde[i] = data[stock]['mavg']['price'] / price

        ###########################
        # Inside of OLMAR (algo 2)
        x_bar = x_tilde.mean()
        
        # market relative deviation
        mark_rel_dev = x_tilde - x_bar

        # Expected return with current portfolio
        exp_return = np.dot(self.b_t, x_tilde)
        weight = self.eps - exp_return

        variability = (np.linalg.norm(mark_rel_dev))**2

        if variability == 0.0:
            step_size = 0 # no portolio update
        else:
            step_size = max(0, weight/variability)

        b = self.b_t + step_size*mark_rel_dev
        b_norm = simplex_projection(b)

        #print b_norm
        self.rebalance_portfolio(data, b_norm)
        
        # Predicted return with new portfolio
        pred_return = np.dot(b_norm, x_tilde)

        # update portfolio
        self.b_t = b_norm

    def rebalance_portfolio(self, data, desired_port):
        #rebalance portfolio
        desired_amount = np.zeros_like(desired_port)
        current_amount = np.zeros_like(desired_port)
        prices = np.zeros_like(desired_port)
        
        if self.init:
            positions_value = self.portfolio.starting_cash
        else:
            positions_value = self.portfolio.positions_value + self.portfolio.cash
        
        for i, stock in enumerate(self.stocks):
            current_amount[i] = self.portfolio.positions[stock].amount
            prices[i] = data[stock].price
        
        desired_amount = np.round(desired_port * positions_value / prices)
        diff_amount = desired_amount - current_amount

        for i, stock in enumerate(self.stocks):
            self.order(stock, diff_amount[i]) #order_stock

def simplex_projection(v, b=1):
    """Projection vectors to the simplex domain

    Implemented according to the paper: Efficient projections onto the
    l1-ball for learning in high dimensions, John Duchi, et al. ICML 2008.
    Implementation Time: 2011 June 17 by Bin@libin AT pmail.ntu.edu.sg
    Optimization Problem: min_{w}\| w - v \|_{2}^{2}
    s.t. sum_{i=1}^{m}=z, w_{i}\geq 0

    Input: A vector v \in R^{m}, and a scalar z > 0 (default=1)
    Output: Projection vector w

    :Example:
    >>> proj = simplex_projection([.4 ,.3, -.4, .5])
    >>> print proj
    array([ 0.33333333, 0.23333333, 0. , 0.43333333])
    >>> print proj.sum()
    1.0

    Original matlab implementation: John Duchi (jduchi@cs.berkeley.edu)
    Python-port: Copyright 2012 by Thomas Wiecki (thomas.wiecki@gmail.com).
    """

    v = np.asarray(v)
    p = len(v)

    # Sort v into u in descending order
    v = (v > 0) * v
    u = np.sort(v)[::-1]
    sv = np.cumsum(u)

    rho = np.where(u > (sv - b) / np.arange(1, p+1))[0][-1]
    theta = np.max([0, (sv[rho] - b) / (rho+1)])
    w = (v - theta)
    w[w<0] = 0
    return w


SyntaxError: invalid syntax (adjustments.py, line 639)

In [2]:
import zipline

class OLMARb(zipline.TradingAlgorithm):
    def initialize(self, eps=1, window_length=5):
        # How aggresive should we rebalance the portfolio
        self.eps = eps
        # How many days to compute the moving average over
        self.window_length = window_length
        # Include mavg transform into simulation. Makes it available
        # in handle_data() below.
        self.add_transform(zipline.transforms.MovingAverage, 'mavg', ['price'],
                           window_length=window_length)

    def handle_data(self, data):
        # Calculate how much each price diverged from its moving average
        for stock in data:
            deviation_from_mavg[stock] = data[stock].price - data[stock].mavg
        
        # Calculate new portfolio allocation
        desired_portfolio = current_portfolio + self.eps * deviation_from_mavg
        
        # Rebalance portfolio
        self.rebalance_portfolio(desired_portfolio)

SyntaxError: invalid syntax (adjustments.py, line 639)

In [3]:
import datetime
import zipline
start = datetime.datetime(2001, 8, 1)
end = datetime.datetime(2013, 2, 1)
data = zipline.utils.load_from_yahoo(stocks=STOCKS, indexes={}, start=start, end=end)


SyntaxError: invalid syntax (adjustments.py, line 639)

In [4]:
data.head()


NameError: name 'data' is not defined

In [5]:
def run_olmar(eps=1, window_length=5):
    olmar = OLMAR(eps=eps, window_length=window_length)
    results = olmar.run(data)
    return results.portfolio_value

In [ ]:
results_eps2 = run_olmar(eps=2)
results_eps2.plot()
results_eps1 = run_olmar(eps=1)
results_eps2.plot()
results_eps1.plot(c='g')